In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier,XGBRFClassifier, XGBRegressor, XGBRFRegressor
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from importlib import reload
from pactools.grid_search import GridSearchCVProgressBar
import sys

training_data = pd.read_csv('2024-introduction-to-data-analysis-hw-5-1/train.csv')
testing_data = pd.read_csv('2024-introduction-to-data-analysis-hw-5-1/test.csv')

training_data.dropna(inplace=True)
training_data.drop(columns=["SEASON_ID", "TEAM_ID", 'GP', 'GS', 'MIN'], inplace=True)
testing_data.drop(columns=["ID", "SEASON_ID", "TEAM_ID"], inplace=True)

label_encoder = LabelEncoder()
training_data["position"] = label_encoder.fit_transform(training_data["position"])

X = training_data.drop(columns=["position"])
y = training_data["position"]

kf = KFold(n_splits=5, shuffle=True, random_state=42)
RANDOM_FOREST_PARAMS = {
    'n_estimators': [50, 100, 200, 300, 400, 500],
    'max_features': [1, 2, 'sqrt', 'log2'],
    'max_depth' : [4, 6, 8, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion' :['gini', 'entropy']
}
model = RandomForestClassifier()

search = RandomizedSearchCV(model, RANDOM_FOREST_PARAMS, n_iter=100, scoring='f1_weighted', n_jobs=-1, cv=kf, random_state=42)
search.fit(X, y)
print(search.best_params_)
model = search.best_estimator_

# model.fit(X, y)

y_pred = model.predict(testing_data)
submission_df = pd.DataFrame()
temp = pd.read_csv('2024-introduction-to-data-analysis-hw-5-1/test.csv')
submission_df["ID"] = temp["ID"]
submission_df["position"] = label_encoder.inverse_transform(y_pred)
print(submission_df)
submission_df.to_csv("test_submission.csv", index=False)

{'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 10, 'criterion': 'entropy'}
        ID position
0        1  Forward
1        2  Forward
2        3    Guard
3        4   Center
4        5   Center
...    ...      ...
1995  1996    Guard
1996  1997    Guard
1997  1998  Forward
1998  1999  Forward
1999  2000    Guard

[2000 rows x 2 columns]
